In [ ]:
from parase_input_package.generate_output import *
from parase_input_package.parase_input import *
from queue import Queue
filepath_out='../output/out_corerouter_v2'
netlist_file_path = '../benchmark/bench2.nl'
gridfile_path  = '../benchmark/bench2.grid'
nets,net_num = parse_netlist(netlist_file_path)
rows,columns,bend_penalty,via_penalty,layer1_gird,layer2_grid = parse_gridfile(gridfile_path)

In [ ]:
def unit_cost_router(rows, columns, layer1_grid, nets):
    routing_table = {}
    for net in nets:
        net_id = net['net_id']
        pin1 = net['pin1']
        pin2 = net['pin2']
        layer1_grid[pin1['x']][pin1['y']] = -1
        layer1_grid[pin2['x']][pin2['y']] = -1
        ## 防止布线在后续的pin上，先将所有的pin标记为-1；
        
    for net in nets:
        net_id = net['net_id']
        pin1 = net['pin1']
        pin2 = net['pin2']
        layer1_grid[pin1['x']][pin1['y']] = 1
        layer1_grid[pin2['x']][pin2['y']] = 1
        path = expand_source_to_target(rows, columns, layer1_grid, pin1, pin2)
        mark_path_on_grid(layer1_grid,path)
        routing_table[net_id] = path
    return routing_table

from queue import Queue

def expand_source_to_target(rows, columns, layer1_grid, source, target):
    queue = Queue()
    visited = set()
    parents = {}

    source_tuple = (source['x'], source['y'], source['layer'])
    target_tuple = (target['x'], target['y'], target['layer'])
    
    queue.put(source_tuple)
    visited.add(source_tuple)

    while not queue.empty():
        current_cell = queue.get()

        if current_cell == target_tuple:
            path = reconstruct_path(source_tuple, target_tuple, parents)
            return path

        neighbors = get_neighbors(rows, columns, current_cell)

        for neighbor in neighbors:
            neighbor_tuple = (neighbor['x'], neighbor['y'], neighbor['layer'])
            if neighbor_tuple not in visited:
                if layer1_grid[neighbor['x']][neighbor['y']] != -1:
                    queue.put(neighbor_tuple)
                    visited.add(neighbor_tuple)
                    parents[neighbor_tuple] = current_cell

    return []

def get_neighbors(rows, columns, cell):
    x, y, layer = cell
    neighbors = []
    if x > 0:
        neighbors.append({'x': x - 1, 'y': y, 'layer': layer})
    if x < columns - 1:
        neighbors.append({'x': x + 1, 'y': y, 'layer': layer})
    if y > 0:
        neighbors.append({'x': x, 'y': y - 1, 'layer': layer})
    if y < rows - 1:
        neighbors.append({'x': x, 'y': y + 1, 'layer': layer})

    return neighbors


def reconstruct_path(source, target, parents):
    path = []
    current = target
    while current != source:
        path.append(current)
        current = parents[current]
    path.append(source)
    path.reverse()
    return path

def mark_path_on_grid(layer1_grid, path):
    for cell in path:
        x, y, _ = cell
        layer1_grid[x][y] = -1